In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
# import tushare as ts
import wrds
import talib as ta
import datetime, time

from scipy import stats
from dateutil.relativedelta import *

In [3]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

测试一下是否有crsp.msenames的权限

In [4]:
db = wrds.Connection(wrds_username='kpu2019')

Loading library list...
Done


In [5]:
db.describe_table('crsp', 'msenames')

Approximately 99086 rows in crsp.msenames.


,name,nullable,type
0,permno,True,DOUBLE PRECISION
1,namedt,True,DATE
2,nameendt,True,DATE
3,shrcd,True,DOUBLE PRECISION
4,exchcd,True,DOUBLE PRECISION
5,siccd,True,DOUBLE PRECISION
6,ncusip,True,VARCHAR(8)
7,ticker,True,VARCHAR(8)
8,comnam,True,VARCHAR(35)
9,shrcls,True,VARCHAR(4)


In [4]:
tic = time.perf_counter()
db = wrds.Connection(wrds_username='kpu2019')
crsp_name = db.raw_sql("""
                        select permno, shrcd, exchcd, namedt, nameendt
                        from crsp.msenames
                        where exchcd between 1 and 3
                        """)
db.close()
toc = time.perf_counter()
print(toc - tic)

Loading library list...
Done
12.0177721


In [9]:
crsp_name.to_csv('crsp_name.csv')

In [12]:
crsp_name.head()

,permno,shrcd,exchcd,namedt,nameendt
0,10000.0,10.0,3.0,1986-01-07,1986-12-03
1,10000.0,10.0,3.0,1986-12-04,1987-03-09
2,10000.0,10.0,3.0,1987-03-10,1987-06-11
3,10001.0,11.0,3.0,1986-01-09,1993-11-21
4,10001.0,11.0,3.0,1993-11-22,2004-06-09


In [5]:
tic = time.perf_counter()
db = wrds.Connection(wrds_username='kpu2019')
test = db.raw_sql("""
                        select permno, shrcd, exchcd, namedt, nameendt
                        from crsp.msenames
                        where exchcd between 1 and 3
                        """)
db.close()
toc = time.perf_counter()
print(toc - tic)

Loading library list...
Done
8.37801989999997


In [7]:
test.shape

(91658, 5)

In [4]:
db = wrds.Connection(wrds_username='kpu2019')
crsp_m = db.raw_sql("""
                      select a.permno, a.permco, a.date, b.shrcd, b.exchcd,
                      a.ret, a.retx, a.shrout, a.prc
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/2017' and '12/31/2017'
                      and b.exchcd between 1 and 3
                      """) 
db.close()

Loading library list...
Done


In [5]:
crsp_m.head()

,permno,permco,date,shrcd,exchcd,ret,retx,shrout,prc
0,10001.0,7953.0,2017-01-31,11.0,2.0,0.007968,0.007968,10520.0,12.65
1,10001.0,7953.0,2017-02-28,11.0,2.0,0.000000,0.000000,10520.0,12.65
2,10001.0,7953.0,2017-03-31,11.0,2.0,0.009881,0.003953,10520.0,12.70
3,10001.0,7953.0,2017-04-28,11.0,2.0,-0.015748,-0.015748,10520.0,12.50
4,10001.0,7953.0,2017-05-31,11.0,2.0,0.016000,0.016000,10520.0,12.70


自己取数的时候一定要限定rows

```
endog, exog, what’s that?
http://www.statsmodels.org/devel/endog_exog.html
```

记得保存下载好之后的数据,最好保存的时候能直接以csv的格式存储,或者hdf

## Compustat Block

```
gvkey
datadate
at -- assets total
pstkl -- preferred stock - liquidating value 优先股 - 清算价值
txditc -- Deferred Taxes and Investment Tax Credit 递延税和投资税收抵免
pstkrv -- Preferred/Preference Stock - Redeemable 首选/优先股 - 可赎回
seq -- Stockholders Equity - Parent
pstk -- Preferred/Preference Stock (Capital) - Total
indfmt -- Industry Format
datafmt
consol -- Level of Consolidation - Company Annual Descriptor
```

In [7]:
db = wrds.Connection(wrds_username='kpu2019')
comp = db.raw_sql("""
                    select gvkey, datadate, at, pstkl, txditc,
                    pstkrv, seq, pstk
                    from comp.funda
                    where indfmt='INDL' 
                    and datafmt='STD'
                    and popsrc='D'
                    and consol='C'
                    and datadate >= '01/01/1959'
                    """)
db.close()

In [43]:
comp.head()

,gvkey,datadate,at,pstkl,txditc,pstkrv,seq,pstk,year
0,001000,1961-12-31,NaN,0.0,0.000,NaN,NaN,NaN,1961
1,001000,1962-12-31,NaN,0.0,NaN,NaN,NaN,0.0,1962
2,001000,1963-12-31,NaN,0.0,0.008,0.0,0.553,0.0,1963
3,001000,1964-12-31,1.416,0.0,0.020,0.0,0.607,0.0,1964
4,001000,1965-12-31,2.310,0.0,0.000,0.0,0.491,0.0,1965


In [18]:
# convert datadate to date fmt (Timestamp)
comp['datadate'] = pd.to_datetime(comp['datadate'])
comp['year'] = comp['datadate'].dt.year

In [52]:
comp.shape

(499996, 10)

In [51]:
# create preferred stock
# 如果可赎回的优先股为空, 则把优先股的清算价值填入, 否则把可赎回的优先股作为优先股
comp['ps'] = np.where(comp['pstkrv'].isnull(), comp['pstkl'], comp['pstkrv'])
# 若此时comp['ps']还为空, 则把总的优先股填入
comp['ps'] = np.where(comp['ps'].isnull(), comp['pstk'], comp['ps'])
# 若此时comp['ps']还为空, 则填0
comp['ps'] = np.where(comp['ps'].isnull(), 0, comp['ps'])

In [53]:
comp['txditc'] = comp['txditc'].fillna(0)

In [55]:
# create book equity
comp['be'] = comp['seq'] + comp['txditc'] - comp['ps']
comp['be'] = np.where(comp['be']>0, comp['be'], np.nan)

`pandas.core.groupby.GroupBy.cumcount`
Essentially this is equivalent to
```python
self.apply(lambda x: pd.Series(np.arange(len(x)), x.index))
```

```
>>> df = pd.DataFrame([['a'], ['a'], ['a'], ['b'], ['b'], ['a']],
...                   columns=['A'])
>>> df
   A
0  a
1  a
2  a
3  b
4  b
5  a
>>> df.groupby('A').cumcount()
0    0
1    1
2    2
3    0
4    1
5    3
dtype: int64
>>> df.groupby('A').cumcount(ascending=False)
0    3
1    2
2    1
3    1
4    0
5    0
dtype: int64
```

In [57]:
# number of years in Compstat
comp = comp.sort_values(by=['gvkey', 'datadate'])
comp['count'] = comp.groupby(['gvkey']).cumcount()

In [60]:
# 提取最终想要的字段
comp = comp[['gvkey','datadate','year','be','count']]

## CRSP Block

```
crsp.msf -- Monthly Stock - Securities

ret -- returns
retx -- returns without dividends
shrout -- shares outstanding
prc -- price or bid/ask average
==============
crsp.msenames -- CRSP Monthly Stock Event - Name History

shrcd -- share code 股票类型代码, 详见
https://wrds-web.wharton.upenn.edu/wrds/query_forms/variable_documentation.cfm?vendorCode=CRSP&libraryCode=CRSPM&fileCode=MSENAMES&id=shrcd
exchcd -- exchange code 详见
https://wrds-web.wharton.upenn.edu/wrds/query_forms/variable_documentation.cfm?vendorCode=CRSP&libraryCode=CRSPM&fileCode=MSENAMES&id=exchcd
```

In [ ]:
db = wrds.Connection(wrds_username='kpu2019')
crsp_m = db.raw_sql("""
                      select a.permno, a.permco, a.date, b.shrcd, b.exchcd,
                      a.ret, a.retx, a.shrout, a.prc
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/1959' and '12/31/2017'
                      and b.exchcd between 1 and 3
                      """) 
db.close()

Loading library list...
Done


In [ ]:
#　change variable format to int
crsp_m[['permco','permno','shrcd','exchcd']] = \
crsp_m[['permco','permno','shrcd','exchcd']].astype(int)

In [ ]:
# Line up date to be end of month
crsp_m['date'] = pd.to_datetime(crsp_m['date'])
# pd.offsets.MonthEnd()
crsp_m['jdate'] = crsp_m['date'] + MonthEnd(0)

```
crsp.msedelist -- CRSP Monthly Stock Event - Delisting

dlret -- delist returns
dlstdt -- delist date
```

In [ ]:
# add delisting return
db = wrds.Connection(wrds_username='kpu2019')
dlret = db.raw_sql("""
                    select permno, dlret, dlstdt 
                    from crsp.msedelist
                    """)
db.close()

In [ ]:
#　change variable format to int
dlret.permno = dlret.permon.astype('int')
# Line up date to be end of month
dlret['dlstdt'] = pd.to_datetime(dlret['dlstdt'])
dlret['jdate'] = dlret['dlstdt'] + MonthEnd(0)

In [ ]:
# Merge normal and delist table
crsp = pd.merge(crsp_m, dlret, how = 'left', on = ['permno', 'jdate'])

In [ ]:
# 清洗数据
crsp['dlret'] = crsp['dlret'].fillna(0)
crsp['ret'] = crsp['ret'].fillna(0)
crsp['retadj'] = (1 + crsp['ret']) * (1 + crsp['dlret']) - 1
crsp['me'] = crsp['prc'].abs() * crsp['shrout'] # calculate market equity
crsp = crsp.drop(['dlret', 'dlstdt', 'prc', 'shrout'], axis=1)
crsp=crsp.sort_values(by=['jdate','permco','me'])

In [ ]:
### Aggregate Market Cap
